In [ ]:
from pathlib import Path
import re
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

CSV_PATH = Path("dataset\\text classifcation.csv") # <— change if needed
RANDOM_STATE = 42
TEST_SIZE = 0.20

In [2]:
# Load
df = pd.read_csv(CSV_PATH)
assert {"text", "label"}.issubset(df.columns), "CSV must contain 'text' and 'label' columns"

# Basic info
print("Rows, Columns:", df.shape)
print("Labels:")
print(df["label"].value_counts())

# Peek a few examples
print(df.head(5))

Rows, Columns: (10000, 2)
Labels:
label
food             2000
tech             2000
sports           2000
politics         2000
entertainment    2000
Name: count, dtype: int64
                                                text   label
0  DEbATinG IF BuRgER🍔 Or bIRYanI is THe TRUe kIn...    food
1  LATEst SMartpHONE bY opeNai dROPpEd tOdAy 🔥 wi...    tech
2  cRicKet COMmeNTArY FelT bIasEd SmH BUT sTILL W...  sports
3  sOfTwaRE upDatE HaD BuGZzZ again 😂 usErs on Tw...    tech
4  soFTwarE updatE Had bugZzz AGAIN 😂 useRs On Tw...    tech


In [4]:
SLANG = {
"lol": "laugh", "lmao": "laugh", "omg": "oh my god", "smh": "shaking my head",
"btw": "by the way", "idk": "i do not know", "imo": "in my opinion", "imho": "in my humble opinion",
"u": "you", "ur": "your", "gr8": "great", "ppl": "people", "pls": "please", "thx": "thanks",
}

EMOJI_REGEX = re.compile(r"[^\x00-\x7F]+") # strip non‑ASCII as a simple emoji proxy
RE_MULTI_SPACE = re.compile(r"\s+")

def normalize_repeats(word: str) -> str:
    """Reduce 3+ repeated characters to 2 (cooool→cool, bugzzzz→bugzz)."""
    return re.sub(r"(.)\1{2,}", r"\1\1", word)

def clean_text(text: str) -> str:
    """Light normalization for noisy short texts."""
    text = str(text).lower()
    text = EMOJI_REGEX.sub(" ", text)
    text = re.sub(r"[~^`|]", " ", text) # drop odd separators

    tokens = []
    for tok in re.findall(r"[a-z0-9]+|[^\w\s]", text, flags=re.UNICODE):
        t = normalize_repeats(tok)
        t = SLANG.get(t, t) # expand slang if present
        tokens.append(t)

    text = " ".join(tokens)
    text = RE_MULTI_SPACE.sub(" ", text).strip()
    return text

# Quick preview of cleaning
preview = df.head(6).copy()
preview["clean"] = preview["text"].apply(clean_text)
print(preview[["text", "clean", "label"]])

                                                text  \
0  DEbATinG IF BuRgER🍔 Or bIRYanI is THe TRUe kIn...   
1  LATEst SMartpHONE bY opeNai dROPpEd tOdAy 🔥 wi...   
2  cRicKet COMmeNTArY FelT bIasEd SmH BUT sTILL W...   
3  sOfTwaRE upDatE HaD BuGZzZ again 😂 usErs on Tw...   
4  soFTwarE updatE Had bugZzz AGAIN 😂 useRs On Tw...   
5  tRiEd thE NeW BUrGER yeStERday OMG it WAs SoOo...   

                                               clean   label  
0  debating if burger or biryani is the true king...    food  
1  latest smartphone by openai dropped today with...    tech  
2  cricket commentary felt biased shaking my head...  sports  
3  software update had bugzz again users on twitt...    tech  
4  software update had bugzz again users on twitt...    tech  
5  tried the new burger yesterday oh my god it wa...    food  


In [5]:
X = df["text"].astype(str)
Y = df["label"].astype(str)

X_train, X_test, y_train, y_test = train_test_split(
X, Y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=Y
)
print(len(X_train), len(X_test))

8000 2000


In [6]:
tfidf = TfidfVectorizer(
preprocessor=clean_text,
ngram_range=(1, 2),
min_df=2,
max_df=0.9,
)

In [7]:
pipe_logreg = Pipeline([
("tfidf", tfidf),
("clf", LogisticRegression(max_iter=200, solver="lbfgs", multi_class="auto")),
])

pipe_nb = Pipeline([
("tfidf", tfidf),
("clf", MultinomialNB()),
])

In [8]:
pipe_logreg.fit(X_train, y_train)
pipe_nb.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.9, min_df=2, ngram_range=(1, 2),
                                 preprocessor=<function clean_text at 0x000001C03E16FEC0>)),
                ('clf', MultinomialNB())])

In [9]:
# Testing and Evaluation
pred_lr = pipe_logreg.predict(X_test)
pred_nb = pipe_nb.predict(X_test)

acc_lr = metrics.accuracy_score(y_test, pred_lr)
acc_nb = metrics.accuracy_score(y_test, pred_nb)

print("=== ACCURACY ===")
print(f"Logistic Regression: {acc_lr:.4f}")
print(f"Multinomial Naive Bayes: {acc_nb:.4f}")

print("\n=== CLASSIFICATION REPORT: Logistic Regression ===")
print(metrics.classification_report(y_test, pred_lr, digits=3))

print("\n=== CLASSIFICATION REPORT: Multinomial Naive Bayes ===")
print(metrics.classification_report(y_test, pred_nb, digits=3))

=== ACCURACY ===
Logistic Regression: 1.0000
Multinomial Naive Bayes: 1.0000

=== CLASSIFICATION REPORT: Logistic Regression ===
               precision    recall  f1-score   support

entertainment      1.000     1.000     1.000       400
         food      1.000     1.000     1.000       400
     politics      1.000     1.000     1.000       400
       sports      1.000     1.000     1.000       400
         tech      1.000     1.000     1.000       400

     accuracy                          1.000      2000
    macro avg      1.000     1.000     1.000      2000
 weighted avg      1.000     1.000     1.000      2000


=== CLASSIFICATION REPORT: Multinomial Naive Bayes ===
               precision    recall  f1-score   support

entertainment      1.000     1.000     1.000       400
         food      1.000     1.000     1.000       400
     politics      1.000     1.000     1.000       400
       sports      1.000     1.000     1.000       400
         tech      1.000     1.000     1.